# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpyhf11jca/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per ten thousand dollars
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in a thousand dollar

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// !wget -O "../data/housing.csv" https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
    try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
}

# Data Ingestion

In [4]:
// TODO: Download housing.csv

let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

let trainPercentage:Float = 0.8
let numTrainRecords = Int(ceil(Float(numRecords) * trainPercentage))
let numTestRecords = numRecords - numTrainRecords

let trainFeatures = Array(dataFeatures[0..<numTrainRecords])
let trainLabels = Array(dataLabels[0..<numTrainRecords])
let testFeatures = Array(dataFeatures[numTrainRecords...])
let testLabels = Array(dataLabels[numTrainRecords...])

print("Training shapes [\(trainFeatures.count) \(trainFeatures[0].count)] [\(trainLabels.count) \(trainLabels[0].count)]")
print("Testing shapes  [\(testFeatures.count) \(testFeatures[0].count)] [\(testLabels.count) \(testLabels[0].count)]")

let xAllTrain = Array(trainFeatures.joined())
let yAllTrain = Array(trainLabels.joined())
let xAllTest = Array(testFeatures.joined())
let yAllTest = Array(testLabels.joined())

let XTrainDeNorm = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTestDeNorm = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

Training shapes [405 13] [405 1]
Testing shapes  [101 13] [101 1]


# Normalize

In [5]:
let mean = XTrainDeNorm.mean(alongAxes: 0)
let std = XTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[  2.0137098,   14.197531,    9.523555, 0.086419754,  0.53213036,   6.3311296,    64.47929,
    4.1678762,    6.832099,   353.68396,    18.03163,   379.84735,   11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.28098345, 0.11449408,  0.7311985,  29.000755,  2.1797554,
    6.544165,  132.14561,  2.2173452,  40.494495,   6.852825]]


In [6]:
let XTrain = (XTrainDeNorm - mean)/std
let XTest = (XTestDeNorm - mean)/std

In [7]:
print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [8]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [9]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [10]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(numTrainRecords) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [11]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(numTrainRecords, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 596.19293, MAE: 22.283003
Epoch 2: MSE: 538.25867, MAE: 21.018812
Epoch 3: MSE: 474.9855, MAE: 19.550512
Epoch 4: MSE: 408.8195, MAE: 17.808851
Epoch 5: MSE: 340.6863, MAE: 15.98984
Epoch 6: MSE: 272.19498, MAE: 13.989948
Epoch 7: MSE: 209.33365, MAE: 11.858542
Epoch 8: MSE: 157.23965, MAE: 9.948924
Epoch 9: MSE: 119.71974, MAE: 8.457142
Epoch 10: MSE: 93.17189, MAE: 7.316224
Epoch 11: MSE: 72.891975, MAE: 6.353826
Epoch 12: MSE: 58.59464, MAE: 5.5594783
Epoch 13: MSE: 48.128765, MAE: 4.9154882
Epoch 14: MSE: 40.11249, MAE: 4.4497347
Epoch 15: MSE: 34.80538, MAE: 4.097543
Epoch 16: MSE: 31.403128, MAE: 3.9274333
Epoch 17: MSE: 28.796242, MAE: 3.7520366
Epoch 18: MSE: 26.862093, MAE: 3.5387268
Epoch 19: MSE: 25.053295, MAE: 3.4584966
Epoch 20: MSE: 23.938978, MAE: 3.2988052
Epoch 21: MSE: 22.981428, MAE: 3.2656627
Epoch 22: MSE: 21.84924, MAE: 3.1455145
Epoch 23: MSE: 21.295015, MAE: 3.1362834
Epoch 24: MSE: 20.342161, MAE: 3.028606
Epoch 25: MSE: 19.592068, MAE: 2.9748673

Epoch 203: MSE: 5.095695, MAE: 1.493724
Epoch 204: MSE: 4.965912, MAE: 1.4875036
Epoch 205: MSE: 5.1447096, MAE: 1.5134832
Epoch 206: MSE: 5.0713477, MAE: 1.5012639
Epoch 207: MSE: 5.0158176, MAE: 1.4932777
Epoch 208: MSE: 4.7853336, MAE: 1.479322
Epoch 209: MSE: 4.7402616, MAE: 1.506598
Epoch 210: MSE: 5.185359, MAE: 1.5026232
Epoch 211: MSE: 4.7926483, MAE: 1.4885743
Epoch 212: MSE: 5.1107416, MAE: 1.4963049
Epoch 213: MSE: 4.827931, MAE: 1.473826
Epoch 214: MSE: 4.933495, MAE: 1.4845603
Epoch 215: MSE: 4.8726616, MAE: 1.4793693
Epoch 216: MSE: 4.886349, MAE: 1.4710826
Epoch 217: MSE: 4.6781588, MAE: 1.4730542
Epoch 218: MSE: 4.8466578, MAE: 1.477794
Epoch 219: MSE: 4.870104, MAE: 1.4716516
Epoch 220: MSE: 4.770494, MAE: 1.4641795
Epoch 221: MSE: 4.5268517, MAE: 1.4736947
Epoch 222: MSE: 4.821713, MAE: 1.4718304
Epoch 223: MSE: 4.697582, MAE: 1.4667645
Epoch 224: MSE: 4.7160707, MAE: 1.4650636
Epoch 225: MSE: 4.7268176, MAE: 1.4464071
Epoch 226: MSE: 4.73833, MAE: 1.4458196
Epoch 227

Epoch 404: MSE: 3.1765022, MAE: 1.2238775
Epoch 405: MSE: 2.9618425, MAE: 1.2099917
Epoch 406: MSE: 2.9515333, MAE: 1.2015872
Epoch 407: MSE: 3.4074125, MAE: 1.2072434
Epoch 408: MSE: 2.8622174, MAE: 1.1622914
Epoch 409: MSE: 3.1732543, MAE: 1.2354379
Epoch 410: MSE: 2.887372, MAE: 1.164552
Epoch 411: MSE: 3.1315558, MAE: 1.2339193
Epoch 412: MSE: 3.2673097, MAE: 1.1956756
Epoch 413: MSE: 2.9652731, MAE: 1.1782377
Epoch 414: MSE: 3.1652238, MAE: 1.2175434
Epoch 415: MSE: 2.933663, MAE: 1.232029
Epoch 416: MSE: 2.897561, MAE: 1.1890689
Epoch 417: MSE: 2.8222604, MAE: 1.1825185
Epoch 418: MSE: 3.3300285, MAE: 1.2634149
Epoch 419: MSE: 3.1406412, MAE: 1.1837922
Epoch 420: MSE: 2.9400213, MAE: 1.1742892
Epoch 421: MSE: 3.0303192, MAE: 1.1690042
Epoch 422: MSE: 2.8937569, MAE: 1.1643794
Epoch 423: MSE: 3.214099, MAE: 1.2597293
Epoch 424: MSE: 3.0254912, MAE: 1.203996
Epoch 425: MSE: 2.8950555, MAE: 1.1747352
Epoch 426: MSE: 2.8308961, MAE: 1.1892842
Epoch 427: MSE: 3.269401, MAE: 1.2159568


# Test

In [12]:
Context.local.learningPhase = .inference

let cheat = model(XTrain)
print(cheat[0], YTrain[0])

let record: [Float] = [-0.4013274553772651,-0.48782210614046656,-1.1729760489283325,-0.2721895900613162,-0.8055945265354896,0.09156749405417394,-1.828902543802867,0.6384789935042571,-0.6351491942719604,0.1472680456555187,-0.7178137893787737,0.2073805740660824,-0.7473489168521552] 
let tfrecord = Tensor<Float>(record).reshaped(to: TensorShape([1, 13]))
let tfcheat = model(tfrecord)
print(tfcheat)

let prediction = model(XTest)
print(prediction[0], YTest[0])

let tmse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numTestRecords)
let tmae = mae(predictions: prediction, truths: YTest)/Float(numTestRecords)

print("MSE: \(tmse), MAE: \(tmae)")

[23.43988] [24.0]
[[24.56447]]
[7.4606285] [5.0]
MSE: 1.0526314, MAE: 0.07994967


# Export

In [13]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [14]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [15]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))

# Sample Record for testing Inferencing on CoreML

In [16]:
print(XTrain[0], YTrain[0])

[ -0.30846816,   0.15054052,   -1.1039964,  -0.30756173,   0.05126577,   0.33352128,
  0.024851447, -0.035726987,  -0.89119065,  -0.43651816,   -1.2319369,   0.42111015,
  -0.93603975] [24.0]
